# Twitter Community Detection and Visualization


For this demo we aim to identify groups of similar users in Twitter network by Attractor, which is a distance dynamics based topology network community detection algorithm. We choose Attractor because it can discover small-sized communities, which is important in this report because we build the Twitter user network based on conversation interaction between users. A conversation may contain only a very small number of users. Based on the community structure identified, we further visualize each community as a "word cloud" to show the underlying high-frequency words discussed by people.


## 1 Attractor 

In [7]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [8]:
class Attractor:
    def __init__(self, G: nx.Graph, cohesion=0.6, max_iter=50, build_j_dist=False, load_neighbors_from_cache=False):
        self.G = G
        self.cohesion = cohesion
        self.max_iter = max_iter
        self.communities = []

        neighbor_cache_path = "./attractor_cache/neighbors.npy"
        need_build_j_dist = 'dist' not in G.edges(data=True).__iter__().__next__()[2].keys()

        if build_j_dist or need_build_j_dist:
            print('Building Jaccard dist matrix...')
            j_dist = list(self._jaccard_dist(ebunch=self.G.edges))
            for (u, v, d) in j_dist:
                self.G[u][v]['dist'] = d
        else:
            print('The graph already has the Jaccard dist attribute.')

        self.common_neighbors = {}
        self.exclusive_neighbors_u = {}
        self.exclusive_neighbors_v = {}
        if load_neighbors_from_cache and os.path.exists(neighbor_cache_path):
            print('Loading neighbors from cache...')
            neighbors = np.load(neighbor_cache_path, allow_pickle=True)
            self.common_neighbors = neighbors[0]
            self.exclusive_neighbors_u = neighbors[1]
            self.exclusive_neighbors_v = neighbors[2]
        else:
            print('Finding common and exclusive neighbors...')
            for u, v in tqdm(self.G.edges()):
                # common neighbors
                if u not in self.common_neighbors.keys():
                    self.common_neighbors[u] = {}
                self.common_neighbors[u][v] = set(nx.common_neighbors(self.G, u, v))

                # exclusive neighbors
                self.exclusive_neighbors_u[u] = set(self.G.neighbors(u)) - self.common_neighbors[u][v]
                self.exclusive_neighbors_v[v] = set(self.G.neighbors(v)) - self.common_neighbors[u][v]

            if not os.path.exists(os.path.dirname(neighbor_cache_path)):
                os.makedirs(os.path.dirname(neighbor_cache_path))
            np.save(neighbor_cache_path, [self.common_neighbors, self.exclusive_neighbors_u, self.exclusive_neighbors_v])

        print('Initialization done.')

    def _jaccard_dist(self, ebunch):
        def predict(G, u, v):
            inter_size = len((set(G[u]) | {u}) & set(G[v]) | {v})
            union_size = len(set(G[u]) | set(G[v]) | {u, v})
            return 1 - inter_size / union_size

        if ebunch is None:
            ebunch = nx.non_edges(G)
        return ((u, v, predict(self.G, u, v)) for u, v in ebunch)

    def _calculate_DI(self, u, v, f):
        tmp = f(1 - self.G[u][v]['dist'])
        DI = -(tmp / self.G.degree(u) + tmp / self.G.degree(v))
        return DI

    def _calculate_CI(self, u, v, f):
        CI = 0
        for x in self.common_neighbors[u][v]:
            CI += 1 / self.G.degree(u) * f(1 - self.G[x][u]['dist']) * (1 - self.G[x][u]['dist']) + 1 / self.G.degree(v) * f(1 - self.G[x][v]['dist']) * (1 - self.G[x][v]['dist'])
        return -CI

    def _calculate_EI(self, u, v, f):
        EI = 0

        def rho(d_xv, cohesion):
            return 1 - d_xv if 1 - d_xv >= cohesion else 1 - d_xv - cohesion

        for x in self.exclusive_neighbors_u[u]:
            EI += 1 / self.G.degree(u) * f(1 - self.G[x][u]['dist']) * rho(self.G[x][u]['dist'], self.cohesion)
        for y in self.exclusive_neighbors_v[v]:
            EI += 1 / self.G.degree(v) * f(1 - self.G[y][v]['dist']) * rho(self.G[y][v]['dist'], self.cohesion)
        return -EI

    def clustering(self, attracted_graph_save_path=None):
        # Interaction
        print('Start interaction...')
        no_finish = True
        iter = 0
        while no_finish and iter < self.max_iter:
            no_finish = False
            iter += 1
            print('Iter {}'.format(iter))
            for (u, v, data) in self.G.edges(data=True):
                d_e = data['dist']
                if 0 < d_e < 1:
                    DI = self._calculate_DI(u, v, np.sin)
                    CI = self._calculate_CI(u, v, np.sin)
                    EI = self._calculate_EI(u, v, np.sin)

                    delta_d_e = DI + CI + EI
                    if delta_d_e != 0:
                        d_e = np.clip(d_e + delta_d_e, 0, 1)
                        self.G[u][v]['dist'] = d_e
                        no_finish = True
        print('End interaction.')

        print('Finding communities.')
        # Find communities
        del_list = []
        for (u, v, data) in self.G.edges(data=True):
            d_e = data['dist']
            if d_e == 1:
                del_list.append((u, v))
        self.G.remove_edges_from(del_list)

        print('Number of clusters: {}.'.format(len(list(nx.connected_components(self.G)))))
        for idx, c in enumerate(sorted(nx.connected_components(self.G), key=len, reverse=True)):
            self.communities.append(c)

        if attracted_graph_save_path is not None:
            print('Saving attracted graph at "{}"'.format(attracted_graph_save_path))
            nx.write_edgelist(self.G, attracted_graph_save_path)

        print('Done.')
        return self.communities

## 2 Performing Community Detection

To perform community detection on Twitter user network, the input to Attractor is the sampled connected component (see "twitter_demo.ipynb") saved in file "subGraphEdges". 

The output is saved in "Community.result"

In [9]:

G = nx.read_edgelist("subGraphEdges", nodetype = int)

# Printing size of the 10 largest community 
attractor = Attractor(G, cohesion=1.3, max_iter=100, load_neighbors_from_cache=True)

# "subGraphEdges.result" is a middle file obtained from "Twitter_Dataset_Analysis.ipynb"
communities = attractor.clustering(attracted_graph_save_path="subGraphEdges.result")

print('Finded clusters:')
first_k = 10
for idx, c in enumerate(communities):
    print('cluster {}, size {}'.format(idx, len(c)))
    if idx >= first_k:
        break
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

Building Jaccard dist matrix...


  0%|          | 377/428640 [00:00<01:53, 3769.79it/s]

Finding common and exclusive neighbors...


100%|██████████| 428640/428640 [01:38<00:00, 4350.33it/s] 


Initialization done.
Start interaction...
Iter 1
Iter 2
Iter 3
Iter 4
Iter 5
Iter 6
Iter 7
Iter 8
Iter 9
Iter 10
Iter 11
Iter 12
Iter 13
Iter 14
Iter 15
Iter 16
Iter 17
Iter 18
Iter 19
Iter 20
Iter 21
Iter 22
Iter 23
Iter 24
Iter 25
Iter 26
Iter 27
Iter 28
Iter 29
Iter 30
Iter 31
Iter 32
Iter 33
Iter 34
Iter 35
Iter 36
Iter 37
Iter 38
Iter 39
Iter 40
Iter 41
Iter 42
Iter 43
Iter 44
Iter 45
Iter 46
Iter 47
Iter 48
Iter 49
Iter 50
Iter 51
Iter 52
Iter 53
Iter 54
Iter 55
Iter 56
Iter 57
Iter 58
Iter 59
Iter 60
Iter 61
Iter 62
Iter 63
Iter 64
Iter 65
Iter 66
Iter 67
Iter 68
Iter 69
Iter 70
Iter 71
Iter 72
Iter 73
Iter 74
Iter 75
Iter 76
Iter 77
Iter 78
Iter 79
Iter 80
Iter 81
Iter 82
Iter 83
Iter 84
Iter 85
Iter 86
Iter 87
Iter 88
Iter 89
Iter 90
Iter 91
Iter 92
Iter 93
Iter 94
Iter 95
Iter 96
Iter 97
Iter 98
Iter 99
Iter 100
End interaction.
Finding communities.
Number of clusters: 3198.
Saving attracted graph at "subGraphEdges.result"
Done.
Finded clusters:
cluster 0, size 2107
cluster 1

## 3 Visulization high-freqency words in each community

In this part, we collect for each user in the network his/her posted tweets about the vaccine, then for each community, we count the freqency of words, and visualize high-freqency word for each community as a word cloud.

The word cloud is saved in dir './word_cloud' as html files.

### 3.1 Node Content (tweets) Collection and Preprocessing

In [6]:
from preprocessing import pipline_processing
from pyecharts.charts import WordCloud

from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
extras_stopwords = set(['vaccine','get','take','make','go','give','know','people','like','one','think','many','say', 'would', 'want','good'])
STOPWORDS = STOPWORDS | extras_stopwords

def generate_node2tweet(raw_file, output_file, G: nx.Graph):
    dataframe = pd.read_csv(raw_file)
    dataframe = dataframe[['user_id', 'tweet',]].dropna(axis=0, how='any', inplace=False)

    node2tweet = {}
    for u in G.nodes():
        tweets = dataframe['tweet'][dataframe['user_id'] == u]
        if tweets.count() > 0:
            node2tweet[u] = pipline_processing(tweets,STOPWORDS)

    np.save(output_file, node2tweet)
    

# change your filepath here 
filename = 'tweets1127_1130.csv' 
generate_node2tweet(filename, "./node2tweet.npy", nx.read_edgelist("./Community.result", nodetype=int))

  

/home/morris/.local/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [7]:
from nltk import FreqDist, word_tokenize


# counting word freqency in a sentence
def count_word_freq(sentence):
    tokenize_sentence = word_tokenize(sentence)
    fdist = FreqDist(word.lower() for word in tokenize_sentence)
    return fdist

# get word frequency in each community
def get_community_word_freq(G: nx.Graph, k, node2tweet, load_from_cache=True):
    
    if load_from_cache:
        fdist_communities = list(np.load('fdist_communities.npy', allow_pickle=True))
    else:
        communities = sorted(nx.connected_components(G), key=len, reverse=True)[:k]
        fdist_communities = []
        node2sent = {}
        
        for k, v in node2tweet.items():
            node2sent[k] = " ".join(v)

        for c in communities:
            fdist_c = FreqDist()
            for u in c:
                fdist_c += count_word_freq(node2sent[u])
            fdist_communities.append(fdist_c)

        np.save('fdist_communities.npy', fdist_communities)

    return fdist_communities


### 3.2 Word Cloud visualization

In [9]:
from pyecharts.charts import WordCloud
from pyecharts import options as opts
from pyecharts.globals import SymbolType
from matplotlib import pyplot as plt
from pyecharts.render import make_snapshot
from snapshot_selenium import snapshot as driver


def visualize_communities(fdists):
    num_words = 30
    shapes = [SymbolType.DIAMOND, SymbolType.RECT, SymbolType.ROUND_RECT, SymbolType.ARROW, SymbolType.TRIANGLE]
    for idx, fdist in enumerate(fdists):

        words = fdist.most_common(num_words)
        w = (WordCloud().add("", words, word_size_range=[20, 100], shape=shapes[0]).set_global_opts(title_opts=opts.TitleOpts(title='Community {}'.format(idx))).render('./word_clouds/Community_{}.html'.format(idx)))
        # You can then check html photos in dir ./word_clouds 

G = nx.read_edgelist("./Community.result", nodetype=int)
node2tweet = np.load("./node2tweet.npy", allow_pickle=True).item()

fdist_communities = get_community_word_freq(G, 9, node2tweet, load_from_cache=False)
visualize_communities(fdist_communities)    
print ('Please check word clouds html in "./word_clouds"')

Please check word clouds html in "./word_clouds"
